In [ ]:
import os
import glob
import random
import re
import copy
import json
from collections import defaultdict

import tqdm
import pydicom
import matplotlib as mpl
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import exposure
import cv2

## Setup

Place kaggle dataset directory in `.`, renamed to `kaggle` such that `./kaggle/test/` exists.

In [ ]:
test = glob.glob("kaggle/test/test/**/*.dcm", recursive=True)

In [ ]:
test_sax = sorted(filter(lambda path: 'sax' in path, test))
test_sax

## Examine DCM File

In [ ]:
dcm = pydicom.dcmread(test_sax[0])

In [ ]:
dcm

In [ ]:
plt.imshow(dcm.pixel_array)

In [ ]:
def scale_pixel_array(pixel_array):
    start, end = np.percentile(pixel_array, (2, 98))
    return exposure.rescale_intensity(pixel_array, in_range=(start, end), out_range="uint16")

In [ ]:
path = random.sample(test_sax, 1)[0]
dcm = pydicom.dcmread(path)
image = scale_pixel_array(dcm.pixel_array)
plt.imshow(image)
print(path)
print(dcm.ImageOrientationPatient)

## Extract Identifiers (Regex)

In [ ]:
all_paths = {}
for t in ["train", "validate", "test"]:
    paths = glob.glob("kaggle/{t}/{t}/**/*.dcm".format(t=t), recursive=True)
    paths = list(sorted(filter(lambda p: 'sax' in p, paths)))
    print(paths[:5])
    all_paths[t] = paths

In [ ]:
re_path = re.compile('kaggle/\w*/\w*/(?P<patient>\d*)/study/sax_(?P<slice>\d*)/.*?-(?P<phase>\d*).dcm')
path = all_paths["train"][0]
match = re_path.match(path)
print(path)
print(match.groups())

In [ ]:
for t, paths in all_paths.items():
    for path in paths:
        if not re_path.match(path):
            raise AssertionError("Invalid path: {}".format(path))
print("All valid")

## Data Cleansing

### Slice Distribution

In [ ]:
_slices = lambda: defaultdict(list)
_phases = lambda: defaultdict(_slices)
patients = defaultdict(_phases)
outliers = defaultdict(list)

duplicates = []
for t, paths in all_paths.items():
    for path in tqdm.tqdm(paths):
        match = re_path.match(path)
        assert(match)
        patient, sid, phase = match.groups()
        if int(sid) in patients[int(patient)][int(phase)]:
            duplicates.append(int(patient))
        patients[int(patient)][int(phase)][int(sid)] = path
outliers["duplicate"] = list(sorted(set(duplicates)))

### Slice Distribution Per Phase

In [ ]:
for patient, phases in patients.items():
    for phase, slices in phases.items():
        print(patient, phase, sorted(slices.keys()))

### Slice Distribution & Outliers

In [ ]:
jumps = []
slice_counts = []

for patient, phases in patients.items():
    # assert slice distribution is identical for all phases
    all_slices = list(phases.values())
    slices = list(sorted(all_slices[0].keys()))
        
    for i, _slices in enumerate(all_slices):
        # ignore patients w/ inconsistent slice distributions between phases
        if len(all_slices[0]) != len(_slices):
            outliers["inconsistent"].append(patient)
            break
            
    slice_counts.append(len(slices))
    prev = slices[0]
    for sid in slices[1:]:
        if sid != prev + 1:
            jumps.append(sid - prev)
            outliers["jumps"].append(patient)
        prev = sid
outliers["jumps"] = sorted(list(set(outliers["jumps"])))

print("Duplicates")
print(outliers["duplicate"])
print()
print("Inconsistent")
print(outliers["inconsistent"])
print()
print("Non-contiguous Slices")
print(outliers["jumps"])

### Distribution of Non-contiguous Slices

In [ ]:
ax = pd.Series(jumps).value_counts().sort_index().plot.bar()

### Number of Slices By Patient

In [ ]:
pd.Series(slice_counts).value_counts().sort_index()

### Filter Outliers

In [ ]:
prime_patients = copy.deepcopy(patients)
for outlier_patients in outliers.values():
    for p in outlier_patients:
        if p in prime_patients:
            del prime_patients[p]

In [ ]:
prime_patients[2][1]

## Export Data To /data/datasets As pngs

In [ ]:
parent = 'KAG'
os.makedirs(parent, exist_ok=True)

for patient, phases in tqdm.tqdm(prime_patients.items()):
    for phase, slices in phases.items():
        if phase == 10:
            for sid, path in slices.items():
                dcm = pydicom.dcmread(path)
                image = scale_pixel_array(dcm.pixel_array)
                dest = 'KAG/KAG_PA{:06d}_PH{:02d}_S{:02d}.png'.format(int(patient), int(phase), int(sid))
                cv2.imwrite(dest, image)

## Export Data To /data/datasets As pngs

In [ ]:
parent = 'KAG'
os.makedirs(parent, exist_ok=True)

for patient, phases in tqdm.tqdm(prime_patients.items()):
    for phase, slices in phases.items():
        if phase == 10:
            for sid, path in slices.items():
                dcm = pydicom.dcmread(path)
                image = scale_pixel_array(dcm.pixel_array)
                dest = 'KAG/KAG_PA{:06d}_PH{:02d}_S{:02d}.png'.format(int(patient), int(phase), int(sid))
                cv2.imwrite(dest, image)